# <img src="https://img.icons8.com/dusk/64/000000/artificial-intelligence.png" style="height:50px;display:inline"> EE 046202 - Technion - Unsupervised Learning & Data Analysis
---

#### <a href="https://www.linkedin.com/in/dan-haramati/">Dan Haramati</a>

## Tutorial 05 - Deep Unsupervised Learning - Vector Quantized VAE (VQ-VAE)
---

### <img src="https://img.icons8.com/bubbles/50/000000/checklist.png" style="height:50px;display:inline"> Agenda
---
* [VAE Reminder](#VAE-Reminder)
* [Motivation and Introduction](#Motivation-&-Introduction)
* [Model](#Model)
  * [Components and Forward Data Flow](#Components-and-Forward-Data-Flow)
  * [Discrete Latent Variables](#Discrete-Latent-Variables)
* [Training](#Training)
  * [Propagating Gradients Through the Dictionary](#Propagating-Gradients-Through-the-Dictionary)
* [Generation](#Generation)
  * [Learning the Prior with Autoregressive Generative Models](#Learning-the-Prior-with-Autoregressive-Generative-Models)
* [Short Summary](#Short-Summary)
* [VQ-VAE 2](#VQ-VAE-2)
  * [Updating the Dictionary with an EMA](#Updating-the-Dictionary-with-an-EMA)
  * [Hierarchical Encoder Architecture](#Hierarchical-Encoder-Architecture)
  * [Hierarchical Generation](#Hierarchical-Generation)
* [Applications](#Applications)
* [Credits](#Credits)


###  <img src="https://img.icons8.com/bubbles/100/000000/qr-code.png" style="height:20px;display:inline"> Variational Autoencoder (VAE) - Reminder
---
The VAE is a deep generative model consisting of a probabalistic encoder and decoder.

Notation:
1. $X$ - the data we want to model (e.g. images of dogs)
2. $z$ - the latent variable (this is the *imagination*, the hidden variable that describes the data, we have seen this before)
3. $p_{\theta}(X)$ - the parameterized probability distribution of the data (e.g. the distribution of all dogs' images in the world). Also, the **evidence**.
4. $p(z)$ - the probability distribution of the latent variables (the source of the imagination, the brain in this case or the distribution of dogs' images feaures/hidden representations). The **prior**.
5. $p_{\theta}(X|z)$ - the parameterized distribution of data generation **given latent variable** (given the features we want the dog to have, the probability of images that satisfy these conditions, turning imagination to real image). The **likelihood** (remember MLE?).
6. $p_{\theta}(z|X)$ - the parameterized distribution of latent variables **given data** (given the image of dog, the probability of latent features that satisfy this image). The **posterior**.

The VAE objective is to maximize the ELBO of the data:
$$ \mathcal{L}_{VAE} = -\mathbb{E}_{q_{\phi}(z|X)}[\log p_{\theta}(X|z)] + D_{KL}[q_{\phi}(z|X)|| p(z)]  $$

Where ${q_{\phi}(z|X)}$ is the Gaussian approximation of the posterior.
The reconstruction loss (first term) is in charge of training the encoder to produce informative latent representations and the decoder to reconstruct the image from them while the KL loss (second term) acts as a regularization term for the encoded latent space. Minimizing the KL ensures the posterior will not be too far from the prior which should enable generating new data by sampling from the simple prior and decoding using the trained decoder.

<img src="assets/vae_lilian_weng.png" height="500">

* <a href="https://lilianweng.github.io/lil-log/2018/08/12/from-autoencoder-to-beta-vae.html#beta-vae">Image by Lilian Weng</a>

The probabilistic encoding induces a **continuous** latent space which allows smooth latent interpolation.

<img src="assets/mnist_interpolation.png" height="100">

<img src="assets/celeb_interpolation.png?raw=1" height="100">

### <img src="https://img.icons8.com/cotton/50/000000/party-baloons.png" style="height:50px;display:inline"> Motivation & Introduction
---
The VQ-VAE was presented in a series of papers by DeepMind researchers:
- ["*Neural Discrete Representation Learning*", Van Den Oord et al. (NeurIPS 2017)](https://arxiv.org/abs/1711.00937)
- ["*Generating Diverse High-Fidelity Images with VQ-VAE-2*", Razavi, Van Den Oord et al. (NeurIPS 2019)](https://arxiv.org/abs/1906.00446)

The main idea is to combine the VAE framework with **discrete** latent representations. \
The motivation for the discrete latent space comes from the modalities which the VQ-VAE aims to model:
- **Language** is discrete in nature.
- **Images** can be described concisely by language.

Therefore, the VQ-VAE latent space aims to capture the essence of the data via a compact discrete representation.

\
The steps for learning a VQ-VAE based generative model are as follows:
1. Learn a discrete latent representation with the VQ-VAE architecture.
2. Learn a prior over the discrete latent space with a generative model of your choice.
3. Sample from the learned prior model and decode using the VQ-VAE decoder to generate new examples.

### Model
---

####  <img src="https://img.icons8.com/color/module" style="height:50px;display:inline"> **Components and Forward Data Flow**

The VQ-VAE model is composed of an encoder $E_{\phi}$, decoder $D_{\theta}$ and a discrete latent embedding space $e=\{e_i\}_{i=1}^{K}$. The parameters of the model are the paramaters of the encoder and decoder neural networks, $\phi$ and $\theta$ respectively, as well as the set of dicrete latent values $e$.

The data is encoded to the latent space with the encoder. The encoded values are then converted to values from the discrete embedding space. This discretized encoding is then passed to the decoder to reconstruct the original data.

<img src="assets/vqvae_discrete.png?raw=1" style="height:300px">

[Image source](https://dl.acm.org/doi/fullHtml/10.1145/3472538.3472584)

####  <img src="https://img.icons8.com/color/grid" style="height:50px;display:inline"> **Discrete Latent Variables**

Define a latent embedding space (also called **dictionary**) $e \in \mathbb{R}^{K \times D} $ where $K$ is the size of the **discrete** latent space and $D$ is the dimensionality of each latent embedding vector $e_i \in \mathbb{R}^{D}$, $i \in 1,2,\dots,K$. Denote $N$ the number of latent variables representing the data, $z_e(x) = [z^c_1, z^c_2, \dots, z^c_N]$ the continous output of the encoder for input $x$ and $z_q(x) = [z_1, z_2, \dots, z_N]$ its corresponding discretization where $z_e(x)_i=z^c_i \in \mathbb{R}^{D}$ and $z_q(x)_i=z_i \in \{e_j\}_{j=1}^{K}$. \
The posterior categorical distribution $q(z|x)$ probabilities are defined as one-hot vectors as follows:
$$ q(z_i=e_k|x) = \begin{cases}
  1 & \mbox{ if $ k=argmin_j\|z_e(x)_i-e_j\|_2 $}\\
  0 & \mbox{ otherwise}
  \end{cases} $$

$q(z_i|x)$ is therefore deterministic and we can write:
$$ z_q(x)_i = e_k, \ \ k=argmin_j\|z_e(x)_i-e_j\|_2 $$

This is simply a nearest neighbor assignment of each latent variable $z_i$ in $z_e(x)$ to $e$.

Note that the output of the encoder $z_e(x)$ and its discretization $z_q(x)$ are both **sequences** of latent variables, where the discrete variables $\{z_i\}_{i=1}^{N}$ all come from the same dictionary. For text or audio, the latent sequence is $1D$ while for images and videos it is a $2D$ and $3D$ grid respectively.

<img src="assets/vqvae_mnist.png?raw=1" height="300">

[Image source](https://keras.io/examples/generative/vq_vae/)

<img src="assets/vqvae_warcraft.png?raw=1" height="300">

[Image source](https://dl.acm.org/doi/fullHtml/10.1145/3472538.3472584)

The above examples illustrate compact discrete latent representations of simple images, where the discrete values correspond to different colors. We may look at this represntation as a downscaling of the original image where the pixel values no longer necessarily correspond to color but to some other discrete latent attribute (e.g., a combination of shape and color or texture).

### <img src="https://img.icons8.com/color/learning" style="height:50px;display:inline"> Training
---

The VQ-VAE loss function consists of 3 terms: reconstruction, vector quantization (VQ) and commitment losses.

$$ \mathcal{L}_{VQ-VAE} = \mathcal{L}_{rec} + \mathcal{L}_{VQ} + \beta\cdot\mathcal{L}_{commit} = -log[p(x|z_q(x))] + \|sg[z_e(x)] - e\|^2_2 + \beta\cdot\|z_e(x) - sg[e]\|^2_2 $$


1. *Reconstruction Loss*: $\mathcal{L}_{rec} = -log[p(x|z_q(x))]$ \
This loss corresponds to the original VAE $ELBO$. Assuming a uniform prior over $z$, the KL term is a constant (exercise: show this), hense we are left with the reconstruction term alone.


2. VQ Loss: $\mathcal{L}_{VQ} = \|sg[z_e(x)] - e\|^2_2$ \
This loss aims to move the embedding vectors $e$ towards the encoder outputs. This term is only used for updating the dictionary, where $s[\cdot]$ is the stop-gradient operator which is defined as the identity in the forward computation and has zero partial derivatives, constraining its operand to be a non-updated constant.


3. *Commitment Loss*: $\mathcal{L}_{commit} = \|z_e(x) - sg[e]\|^2_2$ \
This loss aims to prevent the encoded representations from moving too often between embeddings in order to stabilize training. Here, the stop-gradient operator is applied to the dictionary values. This loss term is weighted by $\beta > 0$.

<img src="https://github.com/lioritan/ee046202-unsupervised-learning-data-analysis/blob/master/assets/vqvae_model.png?raw=1" style="height:300px">

#### **Propagating Gradients Through the Dictionary**

Updating the encoder parameters with respect to the reconstruction loss requires propagating gradients from the decoder to the encoder. This is problematic since the discretization of the encoder output to the dictionary values is a not a differentiable operation.

The solution of the VQ-VAE to this problem is using the **straight-through estimator** for the gradient, which simply copies the gradients from the discretized decoder input to the continous encoder output (see the figure above, left). These gradients contain useful information for how the encoder output has to change to lower the reconstruction loss.

To understand why this is true, consider the figure above (right). If the gradients of the discrete $z_q(x)$ (decoder input) signify that it should move right, moving $z_e(x)$ (encoder output) to the right will allow it to be discretized to a different $z_q(x)$ which is closer to that direction in the next forward pass.

### <img src="https://img.icons8.com/color/lottery.png" style="height:30px;display:inline"> Generation
---


After we have trained a VQ-VAE on our data, we obtain a learned discrete latent representation. We may now use this representation for downstream tasks, one of which is data **generation**.

While with the standard VAE we generate new data by decoding samples from our Gaussian prior, doing the same with the VQ-VAE by sampling from a uniform prior is not likely to produce good results (why?).

We therefore need to **learn the prior** distribution over our latent data representations that the VQ-VAE has converged to. Once we have done that, we can sample from the learned prior and generate new samples by decoding the latent samples with the VQ-VAE decoder (see outline in the figure below).

The VQ-VAE on its own is actually **not a generative model**, and can be more intuitively categorized as a **discrete autoencoder**. To generate new samples, a generative model is trained on the **latent representations** of the data. This makes the generative task easier since modeling a compact representation is more simple than directly modeling high-dimensional noisy data. Once we have a model that is able to generate new latent samples, we convert them to the original high-dimensional form using the VQ-VAE decoder.

<img src="assets/vqvae_gen.png" height="500">

#### **Learning the Prior with Autoregressive Generative Models**

We can essentially use any type of generative model to learn the prior, including Diffusion models, Generative Adversarial Networks (GANs) or even a VAE. Having a discrete latent variable allows using a class of models named **autoregressive** generative models, which we will now describe in slightly more detail.

The VQ-VAE prior is a categorical distribution over the dictionary entries:

$$p(\mathbf{z}) = p(z_1, z_2, \dots, z_N), \ \ z_i \in e_1, \dots, e_K $$

We can modify the distribution to be autoregressive using the chain rule of probability:

$$p(z_1, z_2, \dots, z_N) = p(z_1) \cdot p(z_2, \dots, z_N| z_1) = p(z_1) \cdot p(z_2|z_1) \cdot p(z_3, \dots, z_N| z_1, z_2) = ∏_{i=1}^{N} p(z_i|z_{<i})  $$

This view allows us to fit an autoregressive generative model to the latent data by defining an **order** to the latent variables and parameterizing $p(z_i|z_{<i})$ as a neural network such as an RNN, LSTM, or Transformer. In the case of natural language, there is a natural order which corresponds to the order of the words in the text. In the case of images, one can define an arbitrary order to the pixels, e.g., from top left to bottom right in order of rows. In the first VQ-VAE paper, they use the PixelCNN model to do exactly that.

<img src="assets/vqvae_transformer.png" height="300"> &nbsp; &nbsp; &nbsp; &nbsp; <img src="assets/vqvae_pixelcnn.png?raw=1" height="300">

Image sources: [Transformer Decoder](https://cameronrwolfe.substack.com/p/language-model-training-and-inference) (left), [PixelCNN](https://arxiv.org/pdf/1606.05328.pdf) (right)

The input to the autoregressive model is all the past latent variables $z_{<i} = (z_1, \dots, z_{i-1})$ and the output is a **probability distribution** $p(z_i|z_{<i})$ over the dictionary values $e=\{e_j\}_{j=1}^{K}$.

For more details on autoregressive models and how they are trained see [Deep Learning - ee046211 - Tutorial 7](https://github.com/taldatech/ee046211-deep-learning/blob/main/ee046211_tutorial_07_sequential_tasks_rnn.ipynb).

Once we have trained the model, generation is performed by sampling autoregressively from the output categorical distribution $p(z_i|z_{<i})$:
1. Sample a $z_1$ (e.g., uniformly)
2. For $i=2$ to $N$:
  - Feed $z_{<i}$ as input the model to produce $p(z_i|z_{<i})$
  - Sample from $p(z_i|z_{<i})$ based on a chosen *strategy* (e.g., uniformly from entries with the top $k$ probabilites)
  - Concatente sampled $z_i$ to $z_{<i}$ to produce $z_{<i+1}$

**Note**: the neural network output is deterministic, the *sampling strategy* is in charge of producing random samples from the data distribution.

### <img src="https://img.icons8.com/color/paper.png" style="height:30px;display:inline"> Short Summary
---
The VQ-VAE's major contribution is its compact discrete latent space which is useful for downstream tasks.

One such task is data generation in the latent space which enables faster training and sampling. The process is as follows:

1. Learn a discrete latent representation by training the VQ-VAE on the data.
2. Learn a prior generative model on the latent representations of the data produced with the VQ-VAE encoder.
3. Sample from the learned prior and convert to the original data representation with the VQ-VAE decoder.

### <img src="https://img.icons8.com/color/stairs.png" style="height:30px;display:inline"> VQ-VAE 2
---
The second version of the VQ-VAE introduces two major differences to the model:
1. Replacing the VQ Loss term with an exponential moving average (EMA)
2. Hierarchical encoding and generation

#### **Updating the Dictionary with an EMA**

Instead of learning the dictionary values $e=\{e_j\}_{j=1}^{K}$ by minimizing the VQ loss term $\mathcal{L}_{VQ} = \|sg[z_e(x)] - e\|^2_2$, we can simply update them with an EMA based on the batches of encoder outputs. This works well and is more simple as it does not require differentiation.

Denote:
- $e_i^{(t)}$ the dictionary value $e_i$ after update timestep (batch) $t$
- $n_i^{(t)}$ the number of latent variables assigned to $e_i$ at timestep (batch) $t$
- $\{z_{i,j}^{(t)}\}_{i=1}^{n_i^{(t)}}$ the latent variables assigned to $e_i$ at timestep (batch) $t$

Update Rule:
$$ e_i^{(t)} = \frac{m_i^{(t)}}{N_i^{(t)}} $$

where

$$N_i^{(t)} = \gamma \cdot N_i^{(t-1)} + (1-\gamma) \cdot n_i^{(t)}$$

$$m_i^{(t)} = \gamma \cdot m_i^{(t-1)} + (1-\gamma) \cdot \sum_{j=1}^{n_i^{(t)}} z_{i,j}^{(t)}$$


$0 <\gamma < 1$ is typically chosen such that it is very close to $1$, e.g., $\gamma = 0.99$.

<img src="assets/vqvae2.png?raw=1" style="height:300px">

#### **Hierarchical Encoder Architecture**

VQ-VAE 2 aims to learn a better representation for more complex and high-resolution images. They propose a *hierarchical structure* with the motivation of modeling local information, such as texture, separately from global information such as shape and geometry of objects. This hierarchy is implemented in the encoder alone. The decoder receives all encoded levels as input.

The figure above (left) illustrates the multi-level hierarchical encoder with two levels, each equipped with a **separate latent dictionary**. It is built of multiple encoders, one for each level, and encodes the image in a multi-stage process:
1. The top level encodes the image and is then quantized.
2. The bottom level encodes the image **conditioned** on the quantized top level and is then quantized.

<img src="assets/vqvae2_algo.png?raw=1" height="400">

Conditioning in this context practically means providing another input to the model. This condition (e.g., the top level quantized encoding) is sometimes treated differently than the main input (e.g., the image). There are many conditioning methods, we will not elaborate on them here.

By conditioning each encoded level on the level above it, we **encourage encoding complementary information** about the image in each level.

The figure below presents VQ-VAE 2 reconstructions based on different levels of latent encoding. Notice how each latent level adds extra detail to the reconstruction.

<img src="assets/vqvae_hier_example.png?raw=1" style="height:300px">

#### **Hierarchical Generation**

**Learning the Prior**: Similar to the encoding procedure, the prior is learned in a hierarchical manner. Initially, a generative model is trained on the top level latent representation. In the following stages, separate generative models are trained for each level **conditioned** on the level above it.

**Generating New Data**: Sampling from the prior follows the same steps, starting with sampling the top prior and continuing to sample each lower level conditioned on the sampled level above it. The sampled latents from all levels are then provided as input to the decoder to produce a high resolution image.

<img src="https://img.icons8.com/color/brick.png" style="height:30px;display:inline"> Applications
---

The VQ-VAE and similar models are used in many areas of machine learning including Computer Vision (CV), Natural Language Processing (NLP) and Reinforcement Learning (RL).

Some of the most well known uses of the VQ-VAE include the first version of [DALL-E](https://openai.com/dall-e-3), a text-conditioned generative model for images, and the model-based RL [Dreamer](https://danijar.com/project/dreamerv3/) algorithm.

To see some more interesting applications, check out https://paperswithcode.com/method/vq-vae.

<img src="assets/vqvae_dall-e.png?raw=1" height="300"> &nbsp; &nbsp; <img src="assets/vqvae_dreamer.gif" height="300">

## <img src="https://img.icons8.com/dusk/64/000000/prize.png" style="height:50px;display:inline"> Credits
---
* Tutorial based on ["*Neural Discrete Representation Learning*", Van Den Oord et al. (NeurIPS 2017)](https://arxiv.org/abs/1711.00937) and ["*Generating Diverse High-Fidelity Images with VQ-VAE-2*", Razavi, Van Den Oord et al. (NeurIPS 2019)](https://arxiv.org/abs/1906.00446)
* Icons from <a href="https://icons8.com/">Icon8.com</a> - https://icons8.com
* Datasets from <a href="https://www.kaggle.com/">Kaggle</a> - https://www.kaggle.com/